# SOAP Note Prompt Evaluation
This notebook demonstrates how to iteratively run prompt evaluations and log the results to the Sagemaker-managed MLflow tracking server.

For the examples on how to prepare the evaluation data see `01-Data Pre-Processing.ipynb`

For the annotated example on how to use different utility functions and run evaluation jobs see `02-LLM-Judge-Evaluation.ipynb`

In [1]:
import os

import pandas as pd
from IPython.display import display

from utils.aws import (
    SAGEMAKER_DEFAULT_BUCKET,
    bedrock_runtime_client,
    sagemaker_session,
)
from utils.bedrock import BedrockModel
from utils.evaluation import evaluate_prompt
from utils.metrics import input_tokens_count, output_tokens_count,latency
from utils.metrics.soap_note import completeness, source_grounding
from utils.prompts.soap_note import format_soap_note
from utils._mlflow import set_experiment, set_sagemaker_tracking_server


DATA_DIR = "dataset"

set_sagemaker_tracking_server("a360-mlflow-tracking-server")
mlflow_experiment_name = "SOAP Note Prompt Evaluation"
mlflow_experiment_tags = {
    "task": "prompt-engineering",
    "project": "soap-note-generation",
    "mlflow.note.content": "This experiment evaluates different prompt versions for the SOAP note generation",
}
experiment = set_experiment(mlflow_experiment_name, mlflow_experiment_tags)

/home/sagemaker-user/.conda/envs/soap-notes/lib/python3.12/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## Prompt Definition

In [2]:
PROMPT_VERSION = 3
SOAP_NOTE_GENERATION_PROMPT = """\
You are an expert medical professional tasked with creating a SOAP note based on a transcript of a conversation between a doctor and a patient. A SOAP note is a method of documentation used by healthcare providers that includes four sections: Subjective, Objective, Assessment, and Plan.

You will receive a complete transcript of a doctor-patient conversation.
You must return a detailed SOAP note in JSON format.

<example>

<input>

<transcript>
Good morning, Mrs. Thompson. I'm Doctor. Martinez. Thank you for coming in today. Thank you for seeing me. I understand you're concerned about skin elasticity changes during your pregnancy. Could you tell me more about your current skin care routine? I'm using organic pregnancy safe products that my OB approves. Just basic cleansing and moisturizing. And you mentioned experiencing melasma during your previous pregnancy. Yes, it developed in my second trimester and took several months to fade after delivery. Are you noticing any melasma this pregnancy? Not yet, but I'm being very careful with sun protection. Would you mind if I examined your skin? Not at all. I'm noting mild melasma around the upper cheeks and mild loss of elasticity, which is normal during pregnancy. Your skin hydration appears good. Have you had any skin treatments before? I had a facial about six months ago before getting pregnant. No adverse effects. During pregnancy, we need to be very conservative with treatments. Many common aesthetic procedures aren't recommended until after delivery. However, we can focus on supporting your skin health with appropriate pregnancy safe products and treatments. What would you recommend? I'd suggest a pregnancy safe facial that includes gentle cleansing, hydration, and manual lymphatic drainage to help with any swelling. We can also review your current products and suggest any beneficial additions that are pregnancy safe. And these are definitely safe during pregnancy? Yes. We coordinate with OBGYNs regularly and maintain a strict list of pregnancy safe treatments and products. Would you like me to review these specific ingredients with you? Our pregnancy safe product line includes hyaluronic acid for hydration, vitamin C for antioxidant protection, and niacinamide for skin barrier support. We avoid retinoids, salicylic acid, and other ingredients contraindicated in pregnancy. What about the facial treatment itself? The facial takes about fifty minutes. We use gentle pressure and pregnancy safe positions. Cost is 150 per session, and we recommend monthly treatments during pregnancy to maintain skin health. How soon would I see results? You'll notice immediate improvement in hydration and glow. For elasticity, the benefits are cumulative with regular treatments. The main goal is to support your skin through the pregnancy and minimize any negative changes. And what about after pregnancy? We can schedule consultation about three months postpartum to reassess your skin and discuss additional treatment options that aren't safe during pregnancy. That makes sense. Would you recommend starting the facial now? Yes. Monthly facials for your remaining pregnancy would be beneficial. We can also add LED light therapy, which is safe during pregnancy and helps with skin health. I'd like to try that. When could I start? We could schedule your first treatment for next week. I'll also provide a list of recommended products for you to review with your OB. Would you like to schedule the treatment now? Yes, that would be great. I'll have our coordinator come in to arrange the appointment and review pricing options with you. Do you have any other questions for me? No, I think you've covered everything. I'll see you next week for your first treatment, Mrs. Thompson.
</transcript>

</input>

<output>

<soap_note>
{{
  "subjective": "Mrs. Thompson is concerned about changes in skin elasticity during her pregnancy and reports using organic, pregnancy‑safe cleansing and moisturizing products approved by her OB. She has a history of melasma in her second trimester of her previous pregnancy, which resolved over several months postpartum. She is not currently noticing new melasma but is diligent about sun protection. She had one facial six months ago (pre‑pregnancy) without adverse effects.",
  "objective": "On exam, there is mild melasma on the upper cheeks, mild loss of skin elasticity (consistent with normal pregnancy changes), and good overall skin hydration. No adverse signs or contraindications observed.",
  "assessment": "1. Pregnancy‑associated melasma – mild and currently stable.\\n2. Mild skin elasticity reduction – physiologic change.\\n3. Overall skin health is good with adequate hydration.",
  "plan": "1. Schedule monthly pregnancy‑safe facials incorporating gentle cleansing, targeted hydration, and manual lymphatic drainage.\\n2. Add LED light therapy for enhanced skin support (safe in pregnancy).\\n3. Review and optimize home regimen: continue organic products; emphasize hyaluronic acid (hydration), vitamin C (antioxidant), and niacinamide (barrier support); avoid retinoids, salicylic acid, and other contraindicated ingredients.\\n4. Provide patient with a printed list of recommended products/ingredients to review with her OB.\\n5. First treatment scheduled next week; coordinator to confirm appointment and pricing details.\\n6. Postpartum follow‑up visit at three months after delivery to reassess skin and consider treatments contraindicated during pregnancy."
}}
</soap_note>

</output>

</example>

Instructions:
1. Carefully analyze the following transcript of a conversation between a doctor and a patient:

<transcript>
{transcript}
</transcript>

2. Now, extract all the relevant information for each section of the SOAP note. Follow these guidelines for each section:
    a. Subjective: Include the patient's chief complaint, history of present illness, and any relevant past medical history, family history, or social history mentioned by the patient.
    b. Objective: Note any physical examination findings, vital signs, or test results mentioned by the doctor.
    c. Assessment: Summarize the doctor's diagnosis or differential diagnoses based on the subjective and objective information. Include any medical reasoning or thought process expressed by the doctor.
    d. Plan: List any treatment plans, medications prescribed, further tests ordered, referrals made, or follow-up instructions given by the doctor.

3. Provide your output as a JSON object with four keys: "subjective", "objective", "assessment", and "plan". Each key should contain a string value with the relevant information for that section."""
ASSISTANT_RESPONSE_PREFILL = """\
{
    "subjective": \""""

## Data Preparation

In [3]:
# load evaluation data
eval_data_path = os.path.join(DATA_DIR, "transcripts-plain.csv")
if not os.path.isfile(eval_data_path):
    sagemaker_session.download_data(
        DATA_DIR,
        SAGEMAKER_DEFAULT_BUCKET,
        "prompt-engineering/soap-notes/dataset/transcripts-plain.csv",
    )
eval_df = pd.read_csv(eval_data_path)
eval_df

,transcript
0,"Hi, Ava. I'm Doctor. Bennett. It's great to me..."
1,Good morning. I'm Doctor. Chen. You must be Da...
2,"Good morning, Ms. Cooper. I'm Doctor. Bennett...."
3,"Hello, Jasmine. How have you been? Look, doc. ..."
4,"Hello, I'm Doctor. Patterson. You must be Jenn..."
5,Good morning. I'm Doctor. Chin. You must be Ka...
6,"Hello Mia, I'm Doctor. Harrison. Please come i..."
7,"Good morning, Mrs. Parker. I'm Doctor. Roberts..."
8,"Morning Ms. Davis, I'm Doctor. Warren. What br..."
9,Good morning Ms. Wright. How are you today? Gr...


## Model Definition

In [4]:
inference_config = {"temperature": 0.0}
sonnet = BedrockModel(
    "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    bedrock_runtime_client,
    inf_config=inference_config,
)
models = [sonnet]

## Metrics Definition

In [5]:
aggregations = ["min", "max", "mean", "median"]
judge_metrics = [completeness(aggregations), source_grounding(aggregations)]
operational_metrics = [
    input_tokens_count(aggregations),
    output_tokens_count(aggregations),
    latency(aggregations),
]

## Evaluation

In [6]:
preds_col_name = "soap_note"
eval_results = evaluate_prompt(
    SOAP_NOTE_GENERATION_PROMPT,
    models,
    eval_df,
    sonnet,
    judge_metrics,
    preds_col_name,
    operational_metrics,
    f"prompt-v{PROMPT_VERSION}-",
    response_prefill=ASSISTANT_RESPONSE_PREFILL,
    custom_cols={preds_col_name: format_soap_note},
)

2025/04/17 11:01:14 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.
2025/04/17 11:01:14 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
2025/04/17 11:05:03 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...


🏃 View run prompt-v3-claude-3.7-sonnet at: https://us-east-1.experiments.sagemaker.aws/#/experiments/35/runs/05fb2681de434071a4d88bbe237d6825
🧪 View experiment at: https://us-east-1.experiments.sagemaker.aws/#/experiments/35


In [7]:
for run_name, result in eval_results.items():
    print(f"Evaluation results for run {run_name}:")
    display(result.tables["eval_results_table"].head(3))

Evaluation results for run prompt-v3-claude-3.7-sonnet:


,transcript,outputs,output_tokens,input_tokens,latency_ms,input_tokens_count/score,output_tokens_count/score,latencyMs/score,completeness/score,completeness/justification,source_grounding/score,source_grounding/justification
0,"Hi, Ava. I'm Doctor. Bennett. It's great to me...",Subjective: Ava presents with concerns about h...,419,2485,16281,2485,419,16281,5,## Subjective\nThe subjective section is very ...,4,## Subjective\nThe subjective section accurate...
1,Good morning. I'm Doctor. Chen. You must be Da...,Subjective: David is a Marine Corps enlistee s...,533,2552,12645,2552,533,12645,5,## Subjective\nThe subjective section is very ...,5,## Subjective\nThe subjective section accurate...
2,"Good morning, Ms. Cooper. I'm Doctor. Bennett....",Subjective: Ms. Cooper presents with concerns ...,377,2063,8847,2063,377,8847,5,## Subjective\nThe subjective section comprehe...,4,## Subjective\nThe subjective section accurate...
